In [1]:
class SVM_linear():
    def __init__(self, C=0.1):
        from sklearn.metrics.pairwise import linear_kernel
        from cvxopt import solvers, matrix
        self.C = C
        self.linear_kernel = linear_kernel
        self.matrix = matrix
        self.solvers = solvers
        
    # learn   
    def fit(self, X, y):
        
        # label preprocessing
        a = np.unique(y); c = np.array([1, -1])
        y = np.where(y == a[0], c[0], c[1])
        
        # pre_matrices
        H = self.linear_kernel(X, X); Y = np.outer(y, y)
        Q = np.multiply(Y, H); q = -np.ones(y.shape)
        A = np.array(y.reshape(1, -1), dtype = "float64"); b = 0.0
        ydim = y.shape[0]
        G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
        h_ = np.concatenate((self.C*np.ones(ydim), np.zeros(ydim))); h = h_.reshape(-1, 1)
        
        # matrices for the solver
        matrix = self.matrix
        Q = matrix(Q); q = matrix(q)
        A = matrix(A); b = matrix(b)
        G = matrix(G); h = matrix(h)
        # solver
        solvers = self.solvers
        solvers.options['show_progress']=False
        sol=solvers.qp(P=Q, q=q,G=G,h=h, A=A, b=b)
        
        # alphas threshhold and svs
        alphas = np.array(sol['x']); indx = alphas > 1e-10 
        alpha_sv = alphas[indx]; x_sv = X[indx[:,0],:]; y_sv = y[indx[:,0]]

        # a_k * y_k * x_k
        ayx = np.multiply((alpha_sv*y_sv).reshape(-1, 1), x_sv) 
        # w and b
        w = np.sum(ayx, axis=0)
        b = np.mean(y-np.dot(X,w))
        
        self.w = w; self.b = b; self.c = c; self.a = a
        return self
    
    # predict
    def predict(self, X_test):
        c = self.c; a = self.a
        predict1 = np.sign(np.dot(self.w, X_test.T) + self.b)
        predict2 = np.where(predict1 == c[0], a[0], a[1])
        return predict2
    
    # coefficients
    def coef_(self):
        return self.w, self.b

In [2]:
class plott:
    @staticmethod
    def line(X, w, b):
        xmin, xmax = min(X[:,0])-1, max(X[:,0]+1)
        X_ = np.arange(xmin, xmax, 0.1)
    
        plt.plot(X_,(-w[0]*X_-b)/w[1])
    
    @staticmethod
    def scatter(X, y, m = "o"):
        plt.scatter(X[:,0], X[:,1], c = y, marker = m)

In [ ]:
class contour_plott:
    @staticmethod
    def color(X, y):
        h = .02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
        
        Z = SVM_linear().fit(X, y).predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)